In [91]:
import pandas as pd
import pygwalker as pyg
from matplotlib import pyplot as plt

数据读取

In [92]:
staff_data = pd.read_csv('output.csv',encoding='utf-8')
hospitals_data = pd.read_csv('hospital.csv',encoding='EUC-KR')
elder_data = pd.read_csv('regional_elder.csv',encoding='utf-8')

In [93]:
#modify those fucking column names
def modify_column_name(col):
    return col.split('.')[0]

hospital data preprocessing and renaming the column names

In [94]:
hospital_columns_to_save = ['시도별(1)','2020.1/4','2021.1/4','2022.1/4','2023.1/4']
hospitals_data =hospitals_data[~hospitals_data['시도별(1)'].str.contains('시도별')]
hospitals_data =hospitals_data[hospital_columns_to_save]
columns_to_convert = ['2020.1/4', '2021.1/4', '2022.1/4','2023.1/4']
def remove_commas_and_convert(value):
    try:
        # 移除逗号并转换为整数
        return int(value.replace(',', ''))
    except ValueError:
        # 如果转换失败，返回 NaN
        return pd.NA

# 应用转换函数
columns_to_convert = ['2020.1/4', '2021.1/4', '2022.1/4', '2023.1/4']
for column in columns_to_convert:
    hospitals_data[column] = hospitals_data[column].apply(remove_commas_and_convert)

#将数据第一列搞成index
# hospitals_data.set_index(hospitals_data.columns[0], inplace=True)
pyg.walk(hospitals_data)


Box(children=(HTML(value='\n<div id="ifr-pyg-00061e9ca681d6417xgW4S3iZ2NDenQ5" style="height: auto">\n    <hea…

In [95]:
staff_data.shape

(18, 5)

In [96]:
# 确保两个 DataFrame 具有相同的形状
assert staff_data.shape == hospitals_data.shape, "DataFrames must have the same shape"

# 处理数据列，忽略字符串
def clean_and_divide(df1, df2):
    # 创建结果 DataFrame 的副本
    result = df1.copy()
    
    # 确定数值列的范围（假设第一列是非数值列）
    numeric_columns = df1.columns[1:]  # 从第二列开始假设都是数值列
    
    for column in numeric_columns:
        # 将列转换为字符串类型
        df1[column] = df1[column].astype(str)
        df2[column] = df2[column].astype(str)
        
        # 将逗号移除并转换为数值，无法转换的值变为 NaN
        df1[column] = pd.to_numeric(df1[column].str.replace(',', ''), errors='coerce')
        df2[column] = pd.to_numeric(df2[column].str.replace(',', ''), errors='coerce')
        
        # 执行逐元素除法
        # 为了避免除以零导致的无穷大或错误，使用 `df2[column].replace(0, pd.NA)` 替代 0 以处理除零情况
        result[column] = df1[column] / df2[column].replace(0, pd.NA)
    
    return result

# 使用函数计算
result = clean_and_divide(staff_data, hospitals_data)

# 将索引转移到 DataFrame 的第一列
result = result.reset_index()
result = result.rename(columns={'index': result.columns[0]})
result = result.drop(columns=result.columns[1])  # 删除原来的第一列（索引列）

pyg.walk(result)


KeyError: '2024.1/4'

In [ ]:
# 数字到地名的映射字典
number_to_location = {
    0: '계',
    1: '서울특별시',
    2: '부산광역시',
    3: '대구광역시',
    4: '인천광역시',
    5: '광주광역시',
    6: '대전광역시',
    7: '울산광역시',
    8: '세종특별자치시',
    9: '경기도',
    10: '강원특별자치도',
    11: '충청북도',
    12: '충청남도',
    13: '전라북도',
    14: '전라남도',
    15: '경상북도',
    16: '경상남도',
    17: '제주특별자치도'
}


In [ ]:
staff_data.index = staff_data.index.map(number_to_location)

# 打印转换后的 DataFrame
print("\nDataFrame with location names as index:")
print(staff_data)


DataFrame with location names as index:
         2020.1/4  2021.1/4  2022.1/4  2023.1/4  2024.1/4
계          248635    255715    261238    267572    270661
서울특별시       62934     64309     65523     67079     67150
부산광역시       18745     19107     19473     20030     20191
대구광역시       13606     14060     14425     14742     15115
인천광역시       12516     13066     13311     13984     14398
광주광역시        8944      9064      9145      9348      9371
대전광역시        8624      8768      8856      8927      9004
울산광역시        4340      4436      4536      4638      4724
세종특별자치시       910      1149      1228      1287      1345
경기도         53292     55550     57760     60093     61407
강원특별자치도      6122      6366      6525      6450      6540
충청북도         6543      6622      6769      6798      6947
충청남도         8251      8370      8420      8511      8696
전라북도         9186      9383      9445      9560      9555
전라남도         8261      8431      8549      8504      8466
경상북도         9536      9645    

In [ ]:
print("elder_data index:")
print(elder_data.index)

print("hospitals_data index:")
print(hospitals_data.index)


elder_data index:
Index([1, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21], dtype='int64')
hospitals_data index:
Index(['계', '서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '세종특별자치시', '경기도', '강원특별자치도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도',
       '경상남도', '제주특별자치도'],
      dtype='object', name='시도별(1)')


In [ ]:
hospitals_data.columns

Index(['2020.1/4', '2021.1/4', '2022.1/4', '2023.1/4'], dtype='object')

In [ ]:
elder_data.columns

Index(['성별', '세대구성별', '2020', '2020.1', '2020.2', '2020.3', '2020.4', '2020.5',
       '2020.6', '2021', '2021.1', '2021.2', '2021.3', '2021.4', '2021.5',
       '2021.6', '2022', '2022.1', '2022.2', '2022.3', '2022.4', '2022.5',
       '2022.6', '2023', '2023.1', '2023.2', '2023.3', '2023.4', '2023.5',
       '2023.6'],
      dtype='object')

In [ ]:
#now set the 1st column of elder dataset as the index, and synthesize the doctor dataset same as the elder dataset
#change column index of doctor df
hospitals_data.columns = ['2020', '2021','2022','2023']
#change doctor data index from digits to the location
hospitals_data.index = hospitals_data.index.map(number_to_location)

#change the elder data 1st column into index
elder_data.set_index('행정구역별(시군구)', inplace=True)
# 打印转换后的 DataFrame
print("\ndoctor data with location names as index:")
print(hospitals_data.index)
print("\nelder data with location names as index:")
print(elder_data.index)
print(hospitals_data.index==elder_data.index)


doctor data with location names as index:
Index([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan],
      dtype='float64', name='시도별(1)')

elder data with location names as index:
Index(['행정구역별(시군구)', '전국', '읍부', '면부', '동부', '서울특별시', '부산광역시', '대구광역시',
       '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원특별자치도', '충청북도',
       '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도'],
      dtype='object', name='행정구역별(시군구)')


ValueError: Lengths must match to compare

In [ ]:
#only save the regional info, and only save the data collected elders who older than 60 ages
column_to_save = ['행정구역별(시군구)','2020','2021','2022','2023']
elder_data = elder_data[column_to_save]

#delete the 행정구역별(시군구),읍부,면부,동부
rows_to_drop = [0,2,3,4] 
elder_data = elder_data.drop(index=rows_to_drop)

# 将字符串转换为整数
for col in elder_data.columns[1:]:  # 忽略第一列 '행정구역별(시군구)'
    elder_data[col] = elder_data[col].astype(int)

result = clean_and_divide(elder_data, hospitals_data)
pyg.walk(result)

KeyError: '2020'

In [ ]:
elder_data = elder_data.set_index('행정구역별(시군구)')
print(f'{elder_data.shape=}\n{hospitals_data.shape=}\n')
print(f'{elder_data.columns=}\n{hospitals_data.columns=}\n')
print(f'{elder_data.index=}\n{hospitals_data.index=}\n')



elder_data.shape=(22, 30)
hospitals_data.shape=(18, 4)

elder_data.columns=Index(['성별', '세대구성별', '2020', '2020.1', '2020.2', '2020.3', '2020.4', '2020.5',
       '2020.6', '2021', '2021.1', '2021.2', '2021.3', '2021.4', '2021.5',
       '2021.6', '2022', '2022.1', '2022.2', '2022.3', '2022.4', '2022.5',
       '2022.6', '2023', '2023.1', '2023.2', '2023.3', '2023.4', '2023.5',
       '2023.6'],
      dtype='object')
hospitals_data.columns=Index(['2020.1/4', '2021.1/4', '2022.1/4', '2023.1/4'], dtype='object')

elder_data.index=Index(['행정구역별(시군구)', '전국', '읍부', '면부', '동부', '서울특별시', '부산광역시', '대구광역시',
       '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원특별자치도', '충청북도',
       '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도'],
      dtype='object', name='행정구역별(시군구)')
hospitals_data.index=Index(['계', '서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '세종특별자치시', '경기도', '강원특별자치도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도',
       '경상남도', '제주특별자치도'],
      dtype='obj